In [1]:
!pip install snowflake-connector-python requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 6.1 MB/s eta 0:00:00


# SNOWFLAKES

# WEB SCRAPING

In [2]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [10]:
# Remove previously downloaded Chrome and ChromeDriver if any
!apt-get remove -y google-chrome-stable
!rm /usr/local/bin/chromedriver

# Install the correct Chrome version
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y

# Get the correct version of ChromeDriver for Chrome 129
!wget https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/129.0.6668.58/linux64/chromedriver-linux64.zip
!unzip chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
!chmod +x /usr/local/bin/chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libvulkan1 mesa-vulkan-drivers
Use 'apt autoremove' to remove them.
The following packages will be REMOVED:
  google-chrome-stable
0 upgraded, 0 newly installed, 1 to remove and 49 not upgraded.
After this operation, 365 MB disk space will be freed.
(Reading database ... 123760 files and directories currently installed.)
Removing google-chrome-stable (129.0.6668.70-1) ...
Processing triggers for man-db (2.10.2-1) ...
--2024-09-25 17:24:52--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 64.233.187.93, 64.233.187.136, 64.233.187.91, ...
Connecting to dl.google.com (dl.google.com)|64.233.187.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111860972 (107M) [application/x-debian-package]
Saving to: ‘google-chrome-st

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chrome_options.binary_location = "/usr/bin/google-chrome"

driver = webdriver.Chrome(options=chrome_options)

main_url = "https://www.data.gov.in/catalog/schools-availability-infrastructure-and-facilities-school-management-and-school-category"

def get_catalog_urls(main_url):
    try:
        # Get page source from the Selenium driver
        driver.get(main_url)
        driver.implicitly_wait(10)
        page_source = driver.page_source

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        catalog_urls = []

        # Navigate through the class hierarchy to find the "Data API" links
        containers = soup.find_all('div', class_='container')

        if containers:
            for container in containers:
                rows = container.find_all('div', class_='row')
                for row in rows:
                    cols = row.find_all('div', class_='col-md-6')
                    for col in cols:
                        card = col.find('div', class_='card grid-box')
                        if card:
                            grid_download_box = card.find('div', class_='grid-download-box')
                            if grid_download_box:
                                grid_download_inner = grid_download_box.find('div', class_='grid-download-box-inner')
                                if grid_download_inner:
                                    grid_export_box = grid_download_inner.find('div', class_='grid-export-box')
                                    if grid_export_box:
                                        # Find all "Data API" links within the export box
                                        links = grid_export_box.find_all('a', title="Data API")
                                        for link in links:
                                            catalog_url = link.get('href')
                                            if catalog_url:
                                                # Complete the URL in case it's relative
                                                full_url = f"https://www.data.gov.in{catalog_url.split('#')[0]}"
                                                catalog_urls.append(full_url)
    except Exception as e:
        print(f"Error fetching catalog URLs from {main_url}: {e}")

    return catalog_urls


# Call the function and print the catalog URLs
# catalog_urls = get_catalog_urls(main_url)
# print("Catalog URLs:", catalog_urls)

In [17]:
url = "https://www.data.gov.in/resource/number-schools-availability-infrastructure-and-facilities-school-management-and-2798#api"
api_key = "579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b"

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_endpoint_from_catalog(url, api_key, format="json", limit="50"):
    driver.get(url)

    try:
        # Explicitly wait for the API path element to load (adjust the selector if needed)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "opblock-summary-path"))
        )

        # Once the element is found, proceed with scraping
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        api_endpoints = []

        # Find the relevant container and API paths
        containers = soup.find_all('div', class_='wrapper')
        for container in containers:
            path_span = container.find('span', class_='opblock-summary-path')
            if path_span:
                data_path = path_span.get_text(strip=True)
                full_url = f"https://api.data.gov.in{data_path}?api-key={api_key}&format={format}&limit={limit}"
                api_endpoints.append(full_url)

        return api_endpoints

    except Exception as e:
        # Handle exceptions such as timeout or missing element
        print(f"Error fetching API endpoint: {e}")
        return []

# get_endpoint_from_catalog(url, api_key)

In [ ]:
url2 = "https://www.data.gov.in/resource/number-schools-availability-infrastructure-and-facilities-school-management-and-2783#api"
get_endpoint_from_catalog(url2, api_key)

['https://api.data.gov.in/resource/99749f05-5b1b-44ef-a1ac-5ebbe11a4b3f?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=50']

In [18]:
api_key = "579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b"
def fetch_all_endpoints(catalog_urls, api_key, format="json", limit="50"):
    all_endpoints = []
    for url in catalog_urls:
        try:
            url_with_api = url + "#api"
            endpoints = get_endpoint_from_catalog(url_with_api, api_key, format=format, limit=limit)
            all_endpoints.extend(endpoints)
        except Exception as e:
            print(f"Error fetching endpoints from {url}: {e}")
    return all_endpoints

# fetch_all_endpoints(catalog_urls, api_key)

# STORING WEB SCRAPED DATA TO SNOWFLAKES

In [19]:
import snowflake.connector
import requests
import json

# Step 3: Connect to Snowflake
ctx = snowflake.connector.connect(
    user='MEHEKJAIN',
    password='Yrztypk3NNveFde',
    account='UKCYXTT-ZL80105',
    warehouse='COMPUTE_WH',
    database='TEST',
    schema='PUBLIC',
    role='ACCOUNTADMIN'
)

# Step 4: Create a cursor object to execute SQL
cur = ctx.cursor()

def create_table_in_snowflakes(table_name, cur):
  create_table_query = f"""
  CREATE TABLE IF NOT EXISTS {table_name} (
      academic_year VARCHAR,
      state_code VARCHAR,
      state_name VARCHAR,
      district_code VARCHAR,
      district_name VARCHAR,
      block_code VARCHAR,
      udise_block_name VARCHAR,
      school_category_id VARCHAR,
      school_category_name VARCHAR,
      school_management_id VARCHAR,
      school_management_name VARCHAR,
      location VARCHAR,
      school_type_id VARCHAR,
      school_type VARCHAR,
      total_number_of_schools VARCHAR,
      building VARCHAR,
      complete_medical_checkup VARCHAR,
      computer_available VARCHAR,
      functional_drinking_water VARCHAR,
      drinking_water VARCHAR,
      functional_electricity VARCHAR,
      functional_boy_toilet VARCHAR,
      functional_girl_toilet VARCHAR,
      functional_toilet_facility VARCHAR,
      functional_toilet_and_urinal VARCHAR,
      functional_urinal VARCHAR,
      functional_urinal_boy VARCHAR,
      functional_urinal_girl VARCHAR,
      handwash VARCHAR,
      separate_room_for_headmaster VARCHAR,
      incinerator VARCHAR,
      internet VARCHAR,
      kitchen_garden VARCHAR,
      land_available VARCHAR,
      librarian VARCHAR,
      library_or_reading_corner_or_book_bank VARCHAR,
      medical_checkup VARCHAR,
      newspaper VARCHAR,
      playground VARCHAR,
      rain_water_harvesting VARCHAR,
      ramps VARCHAR,
      solar_panel VARCHAR,
      furniture VARCHAR,
      water_purifier VARCHAR,
      water_tested VARCHAR
  );
  """
  try:
      cur.execute(create_table_query)
      print("Table created successfully.")
  except Exception as e:
      print(f"Error creating table: {e}")

def insert_data_into_snowflake(data, table_name, ctx, cur):
  insert_query = f"""
  INSERT INTO {table_name} (
      academic_year, state_code, state_name, district_code, district_name,
      block_code, udise_block_name, school_category_id, school_category_name,
      school_management_id, school_management_name, location, school_type_id,
      school_type, total_number_of_schools, building, complete_medical_checkup,
      computer_available, functional_drinking_water, drinking_water,
      functional_electricity, functional_boy_toilet, functional_girl_toilet,
      functional_toilet_facility, functional_toilet_and_urinal, functional_urinal,
      functional_urinal_boy, functional_urinal_girl, handwash, separate_room_for_headmaster,
      incinerator, internet, kitchen_garden, land_available, librarian,
      library_or_reading_corner_or_book_bank, medical_checkup, newspaper, playground,
      rain_water_harvesting, ramps, solar_panel, furniture, water_purifier, water_tested
  ) VALUES (
      %s, %s, %s, %s, %s,
      %s, %s, %s, %s, %s, %s,
      %s, %s, %s, %s, %s,
      %s, %s, %s, %s, %s,
      %s, %s, %s, %s, %s,
      %s, %s, %s, %s, %s,
      %s, %s, %s, %s, %s,
      %s, %s, %s, %s, %s,
      %s, %s, %s, %s
  );
  """

  for record in data['records']:
      values = (
          str(record['academic_year']),
          str(record['state_code']),
          str(record['state_name']),
          str(record['district_code']),
          str(record['district_name']),
          str(record['block_code']),
          str(record['udise_block_name']),
          str(record['school_category_id']),
          str(record['school_category_name']),
          str(record['school_management_id']),
          str(record['school_management_name']),
          str(record['location']),
          str(record['school_type_id']),
          str(record['school_type']),
          str(record['total_number_of_schools']),
          str(record['building']),
          str(record['complete_medical_checkup']),
          str(record['computer_available']),
          str(record['functional_drinking_water']),
          str(record['drinking_water']),
          str(record['functional_electricity']),
          str(record['functional_boy_toilet']),
          str(record['functional_girl_toilet']),
          str(record['functional_toilet_facility']),
          str(record['functional_toilet_and_urinal']),
          str(record['functional_urinal']),
          str(record['functional_urinal_boy']),
          str(record['functional_urinal_girl']),
          str(record['handwash']),
          str(record['separate_room_for_headmaster']),
          str(record['incinerator']),
          str(record['internet']),
          str(record['kitchen_garden']),
          str(record['land_available']),
          str(record['librarian']),
          str(record['library_or_reading_corner_or_book_bank']),
          str(record['medical_checkup']),
          str(record['newspaper']),
          str(record['playground']),
          str(record['rain_water_harvesting']),
          str(record['ramps']),
          str(record['solar_panel']),
          str(record['furniture']),
          str(record['water_purifier']),
          str(record['water_tested'])
      )

      # Execute the query with the values
      cur.execute(insert_query, values)
  ctx.commit()

def scrape_and_store(main_url, n_pages, table_name, api_key, ctx, cur, format="json", limit="50"):
    try:
      create_table_in_snowflakes(table_name, cur)
      print("Table created successfully.")
      for cnt in range(72,n_pages):
          print(f"FETCHING DATA FROM PAGE {cnt}")
          current_url = main_url + f"?page={cnt}"
          print(f"CURRENT URL: {current_url}")
          catalog_urls = get_catalog_urls(current_url)
          print("CATALOG URLS:", catalog_urls)

          all_endpoints_in_catalog = fetch_all_endpoints(catalog_urls, api_key, format=format, limit=limit)
          print("ALL ENDPOINTS:", all_endpoints_in_catalog)

          for endpoint in all_endpoints_in_catalog:
              try:
                  response = requests.get(endpoint)
                  if response.status_code == 200:
                      data = response.json()  # Converts JSON response to Python dict
                      insert_data_into_snowflake(data, table_name, ctx, cur)
                      print(f"Data added to table. Rows added: {len(data['records'])}")
                  else:
                      print(f"Error fetching data from API at {endpoint}: Status Code {response.status_code}")
              except requests.exceptions.RequestException as req_err:
                  print(f"Request error for endpoint {endpoint}: {req_err}")
              except Exception as e:
                  print(f"Error processing data from {endpoint}: {e}")

          print("Data insertion complete")
      return all_endpoints_in_catalog

    except Exception as e:
        print(f"Error in scraping or storing process: {e}")
        return []  # Return an empty list or partial data as per your needs

In [20]:
main_url = f"https://www.data.gov.in/catalog/schools-availability-infrastructure-and-facilities-school-management-and-school-category"
api_key = "579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b"
table_name = "INFRASTRUCTURE_AVAILABILITY_AND_FACILITIES"
scrape_and_store(main_url, 200, table_name, api_key, ctx, cur, "json", "200")

Table created successfully.
Table created successfully.
FETCHING DATA FROM PAGE 72
CURRENT URL: https://www.data.gov.in/catalog/schools-availability-infrastructure-and-facilities-school-management-and-school-category?page=72
CATALOG URLS: ['https://www.data.gov.in/resource/number-schools-availability-infrastructure-and-facilities-school-management-and-2786', 'https://www.data.gov.in/resource/number-schools-availability-infrastructure-and-facilities-school-management-and-2762', 'https://www.data.gov.in/resource/number-schools-availability-infrastructure-and-facilities-school-management-and-2757', 'https://www.data.gov.in/resource/number-schools-availability-infrastructure-and-facilities-school-management-and-2752', 'https://www.data.gov.in/resource/number-schools-availability-infrastructure-and-facilities-school-management-and-2766', 'https://www.data.gov.in/resource/number-schools-availability-infrastructure-and-facilities-school-management-and-2784', 'https://www.data.gov.in/resource/

KeyboardInterrupt: 

In [ ]:
# Close the browser
driver.quit()